In [ ]:
#edw dimiourgiete to nevroniko diktio to opoio exei eksis kommatia 
#1) To EmbeddingLayer to opoio exei os apotelesma sto forward (h0ij = Wxij + b,)
#2) Ena Graph Attection Layer ftMHA(htij , L(G))
#3) Ena Feed Forward Layer  ftFF(h˙t+1ij)
#4) Ena Output Layer to opoio meta apo epeksergasia apo ola ta alla layers bgazei to regret sto forward rˆij = WhTij + b
#5) Ola mazi ftiaxnoun to nevroniko diktuo 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class EmbeddingLayer(nn.Module):
    def __init__(self, input_dim, output_dim, num_layers=1):
        super(EmbeddingLayer, self).__init__()
        self.W = nn.Parameter(torch.randn(input_dim, output_dim))  # Learnable weight matrix
        self.b = nn.Parameter(torch.zeros(output_dim))  # Learnable biases
        self.num_layers = num_layers

    def forward(self, x):
        h = x
        for _ in range(self.num_layers):
            h = torch.matmul(h, self.W)
            h += self.b
        return h

class GATLayer(nn.Module):
    def __init__(self, input_dim, output_dim, num_heads=8, concat=True):
        super(GATLayer, self).__init__()
        self.gat = GATConv(input_dim, output_dim, heads=num_heads, concat=concat, dropout=0.6)

    def forward(self, x, edge_index):
        #print(x)
        x = self.gat(x, edge_index)
        return F.elu(x)

class FeedForward(nn.Module):
    def __init__(self, input_size, output_size=512):
        super(FeedForward, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        x = self.fc(x)
        x = F.relu(x)
        return x

    
class Output_layer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Output_layer, self).__init__()
        self.W = nn.Parameter(torch.randn(input_dim, output_dim))  # Learnable weight matrix
        self.b = nn.Parameter(torch.zeros(output_dim))  # Learnable biases
        
    def forward(self, x):
        h0ij = torch.matmul(x, self.W)  # Assuming x has shape (batch_size, num_edges, input_dim)
        h0ij += self.b
        return h0ij
    
class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(GNNModel, self).__init__()
        self.embedding_layer = EmbeddingLayer(input_dim, hidden_dim)
        self.batch_norm1 = nn.BatchNorm1d(hidden_dim)
        self.gat_layers = nn.ModuleList([GATLayer(hidden_dim, hidden_dim // 8, num_heads=8) for _ in range(num_layers)])
        self.feedforward = FeedForward(hidden_dim, hidden_dim)  # Adjusted output_size
        self.batch_norm = nn.BatchNorm1d(hidden_dim)
        self.output_layer = Output_layer(hidden_dim, output_dim)

    def forward(self, x, edge_index1):
        x1 = self.embedding_layer(x)
        #print("After embedding layer:", x1.shape)
        #print("After embedding layer:", edge_index1)
        for layer in self.gat_layers:
            x2 = layer(x1, edge_index1)
            #print("After GAT layer:", x2.shape)
            x1 = x1 + x2

        combined1 = x1
        bn_output1 = self.batch_norm(combined1)
       # print("After first batch norm:", bn_output1.shape)
        
        # Apply feedforward operation
        ff_output = self.feedforward(bn_output1)
        #print("After feedforward layer:", ff_output.shape)
        
        # Add the original input with the feedforward output
        combined = bn_output1 + ff_output
       # print("After adding feedforward output:", combined.shape)
        
        # Apply batch normalization
        bn_output = self.batch_norm(combined)
        #print("After second batch norm:", bn_output.shape)
        
        x = self.output_layer(bn_output)
        #print("After output layer:", x.shape)
        return x